# Abstractive Text Summarization using Deep Learning

In [1]:
from tqdm import tqdm
from bs4 import BeautifulSoup

# Data Science
import numpy as np
import pandas as pd

import re

## Data Preprocessing

In [2]:
data = pd.read_csv('Data/amazon-fine-food-reviews.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [4]:
data = data[:100000]

In [5]:
data = data.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time'], 1)

C:\Users\asus\AppData\Local\Temp/ipykernel_4528/3557384794.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Score','Time'], 1)


In [6]:
def cleaner(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [7]:
def clean(column_name):
    preprocessed_text = []
    for sentence in tqdm(data[column_name].values):
        try:
            sentence = re.sub(r"http\S+", "", sentence)
            sentence = BeautifulSoup(sentence, 'lxml').get_text()
            sentence = cleaner(sentence)
            sentence = re.sub("\S*\d\S*", "", sentence).strip()
            sentence = re.sub('[^A-Za-z]+', ' ', sentence)
            preprocessed_text.append(sentence.strip())
        except:
            preprocessed_text.append('')
    return preprocessed_text

In [8]:
data['Cleaned Summary'] = clean('Summary')
print("Summaries have been cleaned.")

  8%|▊         | 8247/100000 [00:02<00:27, 3309.68it/s]C:\Users\asus\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: "..." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(
100%|██████████| 100000/100000 [00:28<00:00, 3527.22it/s]

Summaries have been cleaned.


In [9]:
data['Cleaned Text'] = clean('Text')
print("Texts have been cleaned.")

100%|██████████| 100000/100000 [00:44<00:00, 2242.17it/s]

Texts have been cleaned.


In [10]:
data.dropna(axis=0,inplace=True)
data.to_csv('Data/Cleaned_Data.csv',index=False)

## Model Building

In [11]:
from attention import AttentionLayer

In [12]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
data = pd.read_csv('Data/Cleaned_Data.csv')